# Домашняя работа по теме "Машинное обучение ранжированию"

В этом ДЗ мы:
- научимся работать со стандартным датасетом для машинного обучения ранжированию [MSLR](https://www.microsoft.com/en-us/research/project/mslr/)
- попробуем применить на практике все то, чему мы научились на семинаре

## Как будет происходить сдача ДЗ

Вам надо:
- форкнуть эту репу
- создать бранч в котором вы дальше будете работать
- реализовать класс Model в этом ноутбуке
- убедиться, что ваша реализация выбивает NDCG@10 выше бейзлайна (см. ниже)
- запушить ваш бранч и поставить Pull Request
- в комментарии написать какой скор вы выбили

В таком случае мы (организаторы):

- счекаутим вашу бранчу
- проверим что ваша реализация действительно выбивает заявленный скор

Предполагается, что и вы, и мы работаем в виртаульном окружении как в семинаре про машинное обучение ранжированию: seminars/7-learning-to-rank/requirements.txt(подробнее про работу с виртуальными окружениями README в корне этой репы).

Оценка:
- За выбитый скор больше **0.507** назначаем **5** баллов, за скор больше (или равно) **0.510** назначаем максимальный балл -- 10 баллов
- Тот из участников кто выбъет самый высокий скор получит еще +10 баллов

При сдаче кода важно помнить о том, что:
- В коде не должно быть захардкоженных с потолка взятых гиперпараметров (таких как число деревьев, learning rate и т.п.) -- обязательно должен быть представлен код который их подбирает!
- Решение должно быть стабильно от запуска к запуску (на CPU) т.е. все seed'ы для генераторов случайных чисел должны быть фиксированы
- Мы (организаторы) будем запускать код на CPU поэтому, даже если вы использовали для подбора параметров GPU, финальный скор надо репортить на CPU

## Пререквизиты

Импортируем все что нам понадобится для дальнейшей работы:

In [1]:
import pathlib
from timeit import default_timer as timer
import copy

import numpy as np
import pandas as pd

from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from hyperopt.pyll import scope

from catboost import datasets, utils, CatBoost, Pool

## Датасет MSLR (Microsoft Learning to Rank)

Загрузим датасет MSLR.

Полный датасет можно скачать с официального сайта: https://www.microsoft.com/en-us/research/project/mslr/

Строго говоря, он состоит их 2х частей:

- основной датасет MSLR-WEB30K -- он содержит более 30 тыс. запросов
- "маленький" датасет MSLR-WEB10K, который содержит только 10 тыс. запросов и является случайным сэмплом датасета MSLR-WEB30K

в этом ДЗ мы будем работать с MSLR-WEB10K, т.к. полная версия датасета может просто не поместиться у нас в RAM (и, тем более, в память видеокарты если мы учимся на GPU)

Будем считать, что мы самостоятельно скачали датасет MSLR-WEB10K с официального сайта, поместили его в папку КОРЕНЬ-ЭТОЙ-РЕПЫ/data/mslr-web10k и раззиповали.

В результате у нас должна получиться следующая структура папок:

In [2]:
!ls /kaggle/input/

mslr-web10k


Заметим, что датасет довольно большой, в распакованном виде он весит 7.7 GB.

Датасет состоит из нескольких фолдов, которые по сути представляют из себя разные разбиения одних и тех же данных на обучающее, валидационное и тестовые множеста.

Дальше мы будем использовать только первый фолд: Fold1.

Заглянем внутрь:

In [3]:
!ls -lh /kaggle/input/mslr-web10k/Fold1

total 1.3G
-rw-r--r-- 1 nobody nogroup 267M Apr 24 14:47 test.txt
-rw-r--r-- 1 nobody nogroup 800M Apr 24 14:48 train.txt
-rw-r--r-- 1 nobody nogroup 261M Apr 24 14:47 vali.txt


Видим, что у нас 3 файла с говорящими названиями, соответсвующими сплитам нашего датасета.

Посмотрим на содержимое одного из файлов:

In [4]:
!head -n 1 /kaggle/input/mslr-web10k/Fold1/train.txt

2 qid:1 1:3 2:3 3:0 4:0 5:3 6:1 7:1 8:0 9:0 10:1 11:156 12:4 13:0 14:7 15:167 16:6.931275 17:22.076928 18:19.673353 19:22.255383 20:6.926551 21:3 22:3 23:0 24:0 25:6 26:1 27:1 28:0 29:0 30:2 31:1 32:1 33:0 34:0 35:2 36:1 37:1 38:0 39:0 40:2 41:0 42:0 43:0 44:0 45:0 46:0.019231 47:0.75000 48:0 49:0 50:0.035928 51:0.00641 52:0.25000 53:0 54:0 55:0.011976 56:0.00641 57:0.25000 58:0 59:0 60:0.011976 61:0.00641 62:0.25000 63:0 64:0 65:0.011976 66:0 67:0 68:0 69:0 70:0 71:6.931275 72:22.076928 73:0 74:0 75:13.853103 76:1.152128 77:5.99246 78:0 79:0 80:2.297197 81:3.078917 82:8.517343 83:0 84:0 85:6.156595 86:2.310425 87:7.358976 88:0 89:0 90:4.617701 91:0.694726 92:1.084169 93:0 94:0 95:2.78795 96:1 97:1 98:0 99:0 100:1 101:1 102:1 103:0 104:0 105:1 106:12.941469 107:20.59276 108:0 109:0 110:16.766961 111:-18.567793 112:-7.760072 113:-20.838749 114:-25.436074 115:-14.518523 116:-21.710022 117:-21.339609 118:-24.497864 119:-27.690319 120:-20.203779 121:-15.449379 122:-4.474452 123:-23.634899 

Видим, что данные лежат в уже знакомом нам по семинару формате:

- В первой колонке лежит таргет (оценка асессора), по 5-балльной шкале релевантности: от 0 до 4 (включительно)
- Во второй колонке лежит ID запроса, по которому можно сгруппировать все оценки документов в рамках одного и того же запроса
- Дальше идет вектор из 128 фичей (таких как значения BM25 и т.п.), их точная природа нам сейчас на важна

В файле qid и все-фичи кодируются в формате КЛЮЧ:ЗНАЧЕНИЕ, напр. 130:116 -- тут 130 это номер фичи, а 116 -- ее значение.

Такой формат в мире машинного обучения часто называют svm light формат (в честь когда-то популярной библиотеки SVM-Light)

Напишем немного вспомогательного кода для загрузки этого датасета:

In [5]:
def generate_column_names(num_features):
    """Generates column names for LETOR-like datasets"""
    columns = ['label', 'qid']
    for i in range(num_features):
        column = f"feature_{i+1}"
        columns.append(column)
    return columns
    
def load_svmlight_file(input_file, max_num_lines=0):
    """Loads dataset split in SVM-Light format"""
    def _parse_field(field):
        parts = field.split(':')
        if len(parts) != 2:
            raise Exception(f"invalid number of parts in field {field}")
        return parts

    num_features = 136
    exp_num_fields = num_features + 2
    num_lines = 0
    X = []
    with open(input_file, 'rt') as f:
        for line in f:
            try:
                num_lines += 1
                                  
                # Parse into fields
                fields = line.rstrip().split(' ')
                num_fields = len(fields)
                if num_fields != exp_num_fields:
                    raise Exception(f"invalid number of fields {num_fields}")
    
                # Parse every field
                x = np.zeros(exp_num_fields, dtype=np.float32)
                label = int(fields[0])
                x[0] = label
                _, qid_str = _parse_field(fields[1])
                qid = int(qid_str)
                x[1] = qid
                for i, field in enumerate(fields[2:]):
                    _, feature_str = _parse_field(field)
                    x[i+2] = float(feature_str)
    
                # Add new object
                X.append(x)
                if num_lines % 50000 == 0:
                    print(f"Loaded {num_lines} lines...")
                if max_num_lines > 0 and num_lines == max_num_lines:
                    print(f"WARNING: stop loading, line limit reached: max_num_lines = {max_num_lines} input_file = {input_file}")
                    break
            except Exception as e:
                raise Exception(f"error at line {num_lines} in {input_file}") from e
    
    # To pandas
    df = pd.DataFrame(X, columns=generate_column_names(num_features))
    print(f"Loaded SVM-Light file {input_file}")
    return df

И теперь загрузим датасет:

In [6]:
fold_dir = pathlib.Path("/kaggle/input/mslr-web10k/Fold1")

df_train = load_svmlight_file(fold_dir.joinpath("train.txt"))
df_valid = load_svmlight_file(fold_dir.joinpath("vali.txt"))
df_test = load_svmlight_file(fold_dir.joinpath("test.txt"))
print(f"Dataset loaded from fold_dir {fold_dir}")

Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded 250000 lines...
Loaded 300000 lines...
Loaded 350000 lines...
Loaded 400000 lines...
Loaded 450000 lines...
Loaded 500000 lines...
Loaded 550000 lines...
Loaded 600000 lines...
Loaded 650000 lines...
Loaded 700000 lines...
Loaded SVM-Light file /kaggle/input/mslr-web10k/Fold1/train.txt
Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded SVM-Light file /kaggle/input/mslr-web10k/Fold1/vali.txt
Loaded 50000 lines...
Loaded 100000 lines...
Loaded 150000 lines...
Loaded 200000 lines...
Loaded SVM-Light file /kaggle/input/mslr-web10k/Fold1/test.txt
Dataset loaded from fold_dir /kaggle/input/mslr-web10k/Fold1


Посмотрим на данные:

In [7]:
print(df_train.head(5))

   label  qid  feature_1  feature_2  feature_3  feature_4  feature_5  \
0    2.0  1.0        3.0        3.0        0.0        0.0        3.0   
1    2.0  1.0        3.0        0.0        3.0        0.0        3.0   
2    0.0  1.0        3.0        0.0        2.0        0.0        3.0   
3    2.0  1.0        3.0        0.0        3.0        0.0        3.0   
4    1.0  1.0        3.0        0.0        3.0        0.0        3.0   

   feature_6  feature_7  feature_8  ...  feature_127  feature_128  \
0        1.0        1.0   0.000000  ...         62.0   11089534.0   
1        1.0        0.0   1.000000  ...         54.0   11089534.0   
2        1.0        0.0   0.666667  ...         45.0          3.0   
3        1.0        0.0   1.000000  ...         56.0   11089534.0   
4        1.0        0.0   1.000000  ...         64.0          5.0   

   feature_129  feature_130  feature_131  feature_132  feature_133  \
0          2.0        116.0      64034.0         13.0          3.0   
1          2

Т.е. теперь мы видим что данные доступны в точно таком же виде, как это было в семинаре.

Проведем небольшой EDA.

Всего у нас 723 тыс. документов в трейне:

In [8]:
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723412 entries, 0 to 723411
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 380.8 MB
None


235 тыс. документов в валидации:

In [9]:
print(df_valid.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235259 entries, 0 to 235258
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 123.8 MB
None


И 241 тыс. документов в тесте:

In [10]:
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241521 entries, 0 to 241520
Columns: 138 entries, label to feature_136
dtypes: float32(138)
memory usage: 127.1 MB
None


Сколько у нас запросов?

In [11]:
num_queries_train = df_train['qid'].nunique()
num_queries_valid = df_valid['qid'].nunique()
num_queries_test = df_test['qid'].nunique()
print(f"Got {num_queries_train} train, {num_queries_valid} valid and {num_queries_test} test queries")

Got 6000 train, 2000 valid and 2000 test queries


## Обучаем модель

Теперь можно приступить непосредственно к обучению модели. 

Объявим класс модели, который надо будем заимлементить в этом ДЗ:

In [12]:
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from hyperopt.pyll import scope
from catboost import CatBoost, Pool

In [13]:
import torch

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [14]:
EVAL_METRIC = 'NDCG:top=10;type=Exp'
DEFAULT_PARAMS = {
    'n_estimators': 1000,            # maximum possible number of trees
    'eval_metric': EVAL_METRIC,    # # metric used for early stopping
    'random_seed': 123,
    'verbose': 10,
    'eta': 0.1,
    'max_bin': 64,
    'max_depth': 4,
    'task_type': 'GPU',
    'loss_function': 'YetiRank'
}

In [15]:
def to_catboost_dataset(df):
    y = df['label'].to_numpy()                       # Label: [0-4]
    q = df['qid'].to_numpy().astype('uint32')        # Query Id
    X = df.drop(columns=['label', 'qid']).to_numpy() # 136 features
    return (X, y, q)

In [16]:
X_train, y_train, q_train = to_catboost_dataset(df_train)
X_test, y_test, q_test = to_catboost_dataset(df_test)
X_valid, y_valid, q_valid = to_catboost_dataset(df_valid)
        
pool_train = Pool(data=X_train, label=y_train, group_id=q_train)
pool_test = Pool(data=X_test, label=y_test, group_id=q_test)
pool_valid = Pool(data=X_valid, label=y_valid, group_id=q_valid)

In [17]:
def objective(params=DEFAULT_PARAMS):
    p = copy.deepcopy(DEFAULT_PARAMS)
    p.update(params)
    
    model = CatBoost(p)
    
    model.fit(pool_train, eval_set=pool_valid, use_best_model=True)
    valid_hat = model.predict(pool_valid)
    
    return -utils.eval_metric(y_valid, valid_hat, 'NDCG:top=10;type=Exp', group_id=q_valid)[0]

In [18]:
class FindBestModel:
    def __init__(self):
        self.best_params = None
        self.space = {
            'n_estimators': scope.int(hp.quniform('n_estimators', low=500, high=3000, q=1)),
            'max_bin': scope.int(hp.quniform('max_bin', low=63, high=257, q=1)),
            'max_depth': scope.int(hp.quniform('max_depth', low=4, high=10, q=1))
        }
        
    def find_best(self):
        self.best_params = fmin(fn=objective, space=self.space, algo=tpe.suggest, max_evals=10, show_progressbar=True)
        
    def get_best(self):
        params = copy.deepcopy(DEFAULT_PARAMS)
        params.update(self.best_params)
        model = CatBoost(params)
        model.fit(pool_train, eval_set=pool_valid, use_best_model=True)
        
        return model

In [ ]:
model = FindBestModel()

start = timer()
model.find_best()
elapsed = timer() - start

print(f"Model fit: elapsed = {elapsed:.3f}")

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

Default metric period is 5 because NDCG
 is/are not implemented for GPU

Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time

Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time



0:	test: 0.2289166	best: 0.2289166 (0)	total: 239ms	remaining: 7m 47s

10:	test: 0.3750282	best: 0.3750282 (10)	total: 567ms	remaining: 1m 40s

20:	test: 0.3961445	best: 0.3961445 (20)	total: 786ms	remaining: 1m 12s

30:	test: 0.4056602	best: 0.4056602 (30)	total: 987ms	remaining: 1m 1s

40:	test: 0.4150889	best: 0.4150889 (40)	total: 1.19s	remaining: 55.4s

50:	test: 0.4216341	best: 0.4216341 (50)	total: 1.38s	remaining: 51.7s

60:	test: 0.4261239	best: 0.4261239 (60)	total: 1.58s	remaining: 49.1s

70:	test: 0.4332881	best: 0.4332881 (70)	total: 1.78s	remaining: 47.4s

80:	test: 0.4402895	best: 0.4402895 (80)	total: 1.98s	remaining: 45.9s

90:	test: 0.4464015	best: 0.4464015 (90)	total: 2.18s	remaining: 44.8s

100:	test: 0.4484985	best: 0.4484985 (100)	total: 2.38s	remaining: 43.8s

110:	test: 0.4514399	best: 0.4514399 (110)	total: 2.58s	remaining: 43s

120:	test: 0.4540463	best: 0.4540463 (120)	total: 2.79s	remaining: 42.3s

130:	test: 0.4562240	best: 0.4562240 (130)	total: 2.98s	rem

Default metric period is 5 because NDCG
 is/are not implemented for GPU

Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time

Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time



0:	test: 0.2872485	best: 0.2872485 (0)	total: 82.9ms	remaining: 44.7s            

10:	test: 0.3954296	best: 0.3954296 (10)	total: 442ms	remaining: 21.3s           

20:	test: 0.4113957	best: 0.4113957 (20)	total: 691ms	remaining: 17.1s           

30:	test: 0.4243399	best: 0.4243399 (30)	total: 941ms	remaining: 15.4s           

40:	test: 0.4376521	best: 0.4376521 (40)	total: 1.19s	remaining: 14.5s           

50:	test: 0.4424690	best: 0.4424690 (50)	total: 1.44s	remaining: 13.8s           

60:	test: 0.4481207	best: 0.4481207 (60)	total: 1.69s	remaining: 13.3s           

70:	test: 0.4527719	best: 0.4527719 (70)	total: 1.94s	remaining: 12.8s           

80:	test: 0.4576632	best: 0.4576632 (80)	total: 2.19s	remaining: 12.4s           

90:	test: 0.4593590	best: 0.4593590 (90)	total: 2.44s	remaining: 12s             

100:	test: 0.4621003	best: 0.4621003 (100)	total: 2.69s	remaining: 11.7s         

110:	test: 0.4640934	best: 0.4640934 (110)	total: 2.94s	remaining: 11.4s         

120:

...

0:	test: 0.3002729	best: 0.3002729 (0)	total: 77.1ms	remaining: 3m 50s           

10:	test: 0.4235125	best: 0.4235125 (10)	total: 500ms	remaining: 2m 15s          

20:	test: 0.4311793	best: 0.4311793 (20)	total: 794ms	remaining: 1m 52s          

30:	test: 0.4451034	best: 0.4451034 (30)	total: 1.09s	remaining: 1m 43s          

40:	test: 0.4526643	best: 0.4526643 (40)	total: 1.38s	remaining: 1m 39s          

50:	test: 0.4578210	best: 0.4578210 (50)	total: 1.68s	remaining: 1m 36s          

60:	test: 0.4608380	best: 0.4608380 (60)	total: 1.97s	remaining: 1m 34s          

70:	test: 0.4653384	best: 0.4653384 (70)	total: 2.26s	remaining: 1m 32s          

80:	test: 0.4670949	best: 0.4670949 (80)	total: 2.55s	remaining: 1m 31s          

90:	test: 0.4692927	best: 0.4692927 (90)	total: 2.83s	remaining: 1m 30s          

100:	test: 0.4727938	best: 0.4727938 (100)	total: 3.11s	remaining: 1m 28s        

110:	test: 0.4758150	best: 0.4758150 (110)	total: 3.4s	remaining: 1m 28s         

120:

Default metric period is 5 because NDCG
 is/are not implemented for GPU

Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time

Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time



0:	test: 0.2347339	best: 0.2347339 (0)	total: 58.3ms	remaining: 49.4s            

10:	test: 0.3895700	best: 0.3895700 (10)	total: 408ms	remaining: 31.1s           

20:	test: 0.4068241	best: 0.4068241 (20)	total: 650ms	remaining: 25.6s           

30:	test: 0.4175376	best: 0.4175376 (30)	total: 867ms	remaining: 22.8s           

40:	test: 0.4306213	best: 0.4306213 (40)	total: 1.09s	remaining: 21.5s           

50:	test: 0.4330863	best: 0.4330863 (50)	total: 1.32s	remaining: 20.6s           

60:	test: 0.4417057	best: 0.4417057 (60)	total: 1.53s	remaining: 19.8s           

70:	test: 0.4457411	best: 0.4457411 (70)	total: 1.75s	remaining: 19.2s           

80:	test: 0.4524752	best: 0.4524752 (80)	total: 1.97s	remaining: 18.7s           

90:	test: 0.4554491	best: 0.4554491 (90)	total: 2.19s	remaining: 18.3s           

100:	test: 0.4580153	best: 0.4580153 (100)	total: 2.41s	remaining: 17.8s         

110:	test: 0.4588933	best: 0.4588933 (110)	total: 2.63s	remaining: 17.5s         

120:

Default metric period is 5 because NDCG
 is/are not implemented for GPU

Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time

Metric NDCG:top=10;type=Exp is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time



0:	test: 0.2345233	best: 0.2345233 (0)	total: 63.5ms	remaining: 2m 54s           

10:	test: 0.3849362	best: 0.3849362 (10)	total: 436ms	remaining: 1m 48s          

20:	test: 0.4039507	best: 0.4039507 (20)	total: 686ms	remaining: 1m 29s          

30:	test: 0.4192386	best: 0.4192386 (30)	total: 920ms	remaining: 1m 20s          

40:	test: 0.4264895	best: 0.4264895 (40)	total: 1.15s	remaining: 1m 16s          

50:	test: 0.4346171	best: 0.4346171 (50)	total: 1.38s	remaining: 1m 13s          

60:	test: 0.4421025	best: 0.4421025 (60)	total: 1.6s	remaining: 1m 10s           

70:	test: 0.4466871	best: 0.4466871 (70)	total: 1.81s	remaining: 1m 8s           

80:	test: 0.4529282	best: 0.4529282 (80)	total: 2.06s	remaining: 1m 7s           

90:	test: 0.4545906	best: 0.4545906 (90)	total: 2.27s	remaining: 1m 6s           

100:	test: 0.4568528	best: 0.4568528 (100)	total: 2.49s	remaining: 1m 5s         

110:	test: 0.4582359	best: 0.4582359 (110)	total: 2.71s	remaining: 1m 4s         

120:

Создадим и применим модель:

In [ ]:
print(model.best_params)

In [ ]:
# Fit
start = timer()
best_model = model.get_best()
elapsed = timer() - start

print(f"Model fit: elapsed = {elapsed:.3f}")

# Predict
y_hat_test = best_model.predict(df_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

Сохраняем модель

In [ ]:
model_file = "/kaggle/working/model.cbm"

# Save model
best_model.save_model(model_file)

In [ ]:
!zip model_file

In [ ]:
model = catboost.CatBoost()
model.load_model(model_file)

Теперь, имея предикты, можно посчитать метрики качества:

In [ ]:
def compute_metrics(y_true, y_hat, q):
    # List of metrics to evaluate
    eval_metrics = ['NDCG:top=10;type=Exp']
    
    for eval_metric in eval_metrics:
        scores = utils.eval_metric(y_true, y_hat, eval_metric, group_id=q)
    
        # Print scores
        print(f"metric = {eval_metric} score = {scores[0]:.3f}")

# Get test targets and groups
y_test = df_test['label'].to_numpy()
q_test = df_test['qid'].to_numpy().astype('uint32')
    
# Compute metrics on test
compute_metrics(y_test, y_hat_test, q_test)

Ожидаем, что ваша модель покажет результаты выше бейзлайна!